In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
data_file = "/workspaces/Data-Engineering-Zoomcamp/green_tripdata_2025-11.parquet"
df = pd.read_parquet(data_file)

df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  int32         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  object        
 4   RatecodeID             41343 non-null  float64       
 5   PULocationID           46912 non-null  int32         
 6   DOLocationID           46912 non-null  int32         
 7   passenger_count        41343 non-null  float64       
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  t

# Question 3

In [5]:
# Filter the dataset to the required date range
filtered_df = df[
    (df["lpep_pickup_datetime"] >= "2025-11-01") & 
    (df["lpep_pickup_datetime"] < "2025-12-01")
]

In [6]:
# Count trips with trip_distance <= 1 mile
short_trips_count = filtered_df[filtered_df["trip_distance"] <= 1].shape[0]

print(f"Number of trips with trip_distance <= 1 mile: {short_trips_count}")

Number of trips with trip_distance <= 1 mile: 8007


# Question 4

In [7]:
# filter trips with trip_distance < 100
filtered_df_1 = df[df["trip_distance"] < 100]

# add a new column for the date part of the pickup time
filtered_df_1["pickup_date"] = pd.to_datetime(filtered_df_1["lpep_pickup_datetime"]).dt.date

# find the trip with the longest distance for each day
longest_trip_per_day = filtered_df_1.groupby("pickup_date")["trip_distance"].max().reset_index()

# find the day with the overall longest trip distance
longest_trip_day = longest_trip_per_day.sort_values(by="trip_distance", ascending=False).iloc[0]

print(f"The pick-up day with the longest trip distance is: {longest_trip_day['pickup_date']}")
print(f"Longest trip distance on that day: {longest_trip_day['trip_distance']} miles")


The pick-up day with the longest trip distance is: 2025-11-14
Longest trip distance on that day: 88.03 miles


/tmp/ipykernel_3047/3302629122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_1["pickup_date"] = pd.to_datetime(filtered_df_1["lpep_pickup_datetime"]).dt.date


# Question 5

In [8]:
data_file_1 = "/workspaces/Data-Engineering-Zoomcamp/taxi_zone_lookup.csv"
zones = pd.read_csv(data_file_1)

In [9]:
zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [10]:
zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [11]:
# filter trips for nov 18, 2025 
filtered_df_2 = df[
    (pd.to_datetime(df["lpep_pickup_datetime"]).dt.date == pd.to_datetime("2025-11-18").date())
]

# group trips by pickup location and sum the total amount 
pickup_totals = filtered_df_2.groupby("PULocationID")["total_amount"].sum().reset_index()

# merge with zone names using taxi_zone lookup.csv
pickup_totals = pickup_totals.merge(zones, left_on="PULocationID", right_on="LocationID")

# zones witht the highest total amount 
highest_total = pickup_totals.sort_values(by="total_amount", ascending=False).iloc[0]

print(f"Pickup zone with the highest total_amount on 2025-11-18: {highest_total['Zone']}")
print(f"Total amount: {highest_total['total_amount']}")

Pickup zone with the highest total_amount on 2025-11-18: East Harlem North
Total amount: 9281.92


# Question 6

In [14]:
# merge the trip data with zone names for pickup and drop off
df_1 = df.merge(zones, left_on="PULocationID", right_on="LocationID", suffixes=("", "_pickup"))
df_1 = df_1.merge(zones, left_on="DOLocationID", right_on="LocationID", suffixes=("", "_dropoff"))

# filter trips in novemnber 2025 and where pickup zone = "East Harlem North"
df_filtered_3 = df_1[
    (pd.to_datetime(df_1["lpep_pickup_datetime"]).dt.month == 11) &
    (pd.to_datetime(df_1["lpep_pickup_datetime"]).dt.year == 2025) &
    (df_1["Zone"] == "East Harlem North")
]

# group by drop off zone and find the maximum tip amount 
largest_tip = df_filtered_3.groupby("Zone_dropoff")["tip_amount"].max().reset_index()

# find the dropoff zone with the highest tip
highest_tip_zone = largest_tip.sort_values(by="tip_amount", ascending=False).iloc[0]

print(f"The drop-off zone with the largest tip is: {highest_tip_zone['Zone_dropoff']}")
print(f"Tip amount: {highest_tip_zone['tip_amount']}")

The drop-off zone with the largest tip is: Yorkville West
Tip amount: 81.89
